[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/openlayer-ai/openlayer-python/blob/main/examples/tracing/google-adk/google_adk_tracing.ipynb)

# Google ADK Tracing with Openlayer

This notebook demonstrates how to trace Google Agent Development Kit (ADK) agents with Openlayer.

## Prerequisites

Install the required packages:
```bash
pip install openlayer google-adk wrapt
```


## Setup

First, configure your Openlayer credentials and Google Cloud credentials:


In [ ]:
import os

# Openlayer configuration
os.environ["OPENLAYER_API_KEY"] = "your-api-key-here"
os.environ["OPENLAYER_INFERENCE_PIPELINE_ID"] = "your-pipeline-id-here"

# Google AI API configuration (Option 1: Using Google AI Studio)
# Get your API key from: https://aistudio.google.com/apikey
os.environ["GOOGLE_API_KEY"] = "your-google-ai-api-key-here"

# Google Cloud Vertex AI configuration (Option 2: Using Google Cloud)
# Uncomment these if you're using Vertex AI instead of Google AI
# os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "path/to/your/service-account-key.json"
# os.environ["GOOGLE_CLOUD_PROJECT"] = "your-project-id"
# os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"


## Enable Google ADK Tracing

Enable tracing before creating any agents. This patches Google ADK globally to send traces to Openlayer:


In [ ]:
from openlayer.lib.integrations import trace_google_adk

# Enable tracing (must be called before creating agents)
trace_google_adk()

## Example 1: Basic Agent with LLM Calls

Create a simple agent that responds to user queries:


In [ ]:

from google.genai import types
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService

# Setup constants
APP_NAME = "openlayer_demo"
USER_ID = "user_123"
SESSION_ID = "session_123"

# Create session service (shared across examples)
session_service = InMemorySessionService()

# Create a basic agent
agent = LlmAgent(
    model="gemini-2.0-flash-exp",
    name="Assistant",
    instruction="You are a helpful assistant. Provide concise and accurate responses."
)

# Create runner
runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define async function to run the agent
async def run_basic_agent():
    # Create session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=USER_ID,
        session_id=SESSION_ID
    )
    
    # Run the agent
    query = "What is the capital of France?"
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    # Process events and get response
    async for event in runner.run_async(user_id=USER_ID, session_id=SESSION_ID, new_message=content):
        if event.is_final_response() and event.content:
            final_answer = event.content.parts[0].text.strip()

# Run the async function
await run_basic_agent()

## Example 2: Agent with Tools/Functions

Create an agent with custom tools that can be called during execution:


In [ ]:
import os


# Define custom tools as regular Python functions
def get_weather(city: str) -> str:
    """Retrieves the current weather report for a specified city.
    
    Args:
        city: The name of the city for which to retrieve the weather report.
    
    Returns:
        str: Weather report or error message.
    """
    if city.lower() == "san francisco":
        return "The weather in San Francisco is sunny with a temperature of 72°F (22°C)."
    else:
        return f"Sorry, weather information for '{city}' is not available."

def calculate(expression: str) -> str:
    """Evaluates a mathematical expression.
    
    Args:
        expression: A mathematical expression to evaluate.
    
    Returns:
        str: Calculation result or error message.
    """
    try:
        result = eval(expression)
        return f"The result is {result}"
    except Exception as e:
        return f"Error: {str(e)}"

# Use different session IDs for tool agent
TOOL_USER_ID = "user_456"
TOOL_SESSION_ID = "session_456"

# Create agent with tools (pass functions directly)
tool_agent = LlmAgent(
    model="gemini-2.0-flash-exp",
    name="ToolAgent",
    instruction="You are a helpful assistant with access to weather and calculation tools. Use them when appropriate.",
    tools=[get_weather, calculate]
)

# Create runner for tool agent (reuse the session_service)
tool_runner = Runner(
    agent=tool_agent,
    app_name=APP_NAME,
    session_service=session_service
)

# Define async function to run the tool agent
async def run_tool_agent():
    # Create session
    await session_service.create_session(
        app_name=APP_NAME,
        user_id=TOOL_USER_ID,
        session_id=TOOL_SESSION_ID
    )
    
    # Run the agent with a query that requires tool use
    query = "What's the weather in San Francisco? Also, what is 15 * 24?"
    content = types.Content(role='user', parts=[types.Part(text=query)])
    
    # Process events and get response
    async for event in tool_runner.run_async(
        user_id=TOOL_USER_ID,
        session_id=TOOL_SESSION_ID,
        new_message=content
    ):
        if event.is_final_response() and event.content:
            final_answer = event.content.parts[0].text.strip()

# Run the async function
await run_tool_agent()


## View Traces in Openlayer

After running these examples, you can view the traces in your Openlayer dashboard:

1. Go to https://app.openlayer.com
2. Navigate to your inference pipeline
3. View the traces tab to see:
   - Agent execution steps
   - LLM calls with token counts
   - Tool executions with inputs and outputs
   - Latency for each operation
   - Complete execution hierarchy


## Disable Tracing

When you're done, you can disable tracing to restore ADK's default behavior:


In [ ]:
from openlayer.lib.integrations import unpatch_google_adk

# Disable tracing
unpatch_google_adk()